# Data Cleaning Pipeline

**Notebook: Emmanuel Contreras-Campana, Ph.D.**

This notebook outlines the data cleaning and transformations that were performed on the client company's data for the purposes of the analysis.

## Load Libraries

In [1]:
# Import common python library
import os
import psycopg2
import pandas as pd

# Import urlib library
from urllib import parse

# Import sqlalchemy library
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

# Import user created library
from dskit import *

# Connect to PostgreSQL

In [2]:
parse.uses_netloc.append('postgres')

path_1 = 'URL PATH PART 1'
path_2 ='URL PATH PART 2'

url = parse.urlparse(path_1+path_2)

# set up the connection
dbname = url.path[1:]
username = url.username
password = url.password
host = url.hostname
port = url.port

con = psycopg2.connect(database=dbname, user=username, 
                       password=password, host=host, port=port)

# Load Data

The data is stored on a Heroku PostgreSQL database which we ingest into this notebook and proceed to explore it.

In [3]:
# items sql table
sql_query = """
SELECT *
FROM items
"""

items = pd.read_sql_query(sql_query, con)

In [4]:
# rental_items sql table
sql_query = """
SELECT *
FROM rental_items
"""

rental_items = pd.read_sql_query(sql_query, con)

### Drop Irrelevant Features

We drop all columns that are not relevant for identifying inventory trends.

In [5]:
items.drop(['title', 'description','main_image_id', 'updated_at',
            'main_image_id', 'user_id', 'slug', 'sku', 'fit_description',
            'removed', 'maintenance', 'approved', 'approved_by_id',
            'approved_at', 'year_purchased', 'listing_type', 'neighborhood',
            'delivery_option', 'promoted', 'status', 'time_zone', 'rack',
            'slot', 'review_count', 'review_rating', 'review_fit',
            'admin_notes', 'purchased_fake', 'photo_status',
            'condition', 'model', 'for_sale', 'sale_price',
            'only_for_sale', 'oversized', 'virtual_try_url',
            'designer_id'], axis=1, inplace=True)

In [6]:
rental_items.drop(['id', 'status', 'created_at', 'updated_at', 'fit_return',
                   'refunded', 'fit_return_reason', 'fit_return_notes'],
                  axis=1, inplace=True)

In [7]:
# Rename rent_per_week to rental_price

items.rename({'rent_per_week': 'rental_price'}, axis='columns', inplace=True)

### Produce Orders Table

We join the items table to the rental items tables to have a more meaningful table which can be used to explore trends in the fashion items ordered.

In [8]:
orders = rental_items.merge(items, how='left', left_on='item_id',
                            right_on='id').drop('id', axis=1)

### Rental Counts

From the orders table we can determine the rental count of each item. This information can be used, for example, to learn how often an item is rented during its listing lifetime.

In [9]:
# calculate the number of rentals per item

rental_count = orders.groupby('item_id',axis=0, as_index=False)\
                     .count().rename(columns={'rental_id':'rental_count'})

rental_count = rental_count[['item_id', 'rental_count']]

We now include the rental count information to the items table. Some inventory may have been rented out several times while other items may have never been rented out at any point in time.

In [10]:
# merge rental count with items

items = items.merge(rental_count, how='left',
                    left_on='id', right_on='item_id').drop('item_id', axis=1)

# items that were never rented are given a rental count of zero
items['rental_count'] = items['rental_count'].fillna(value=0)

### Rental Revenue

An interesting quantity to have is the rental revenue of each item which may then be used later to evaluate the improvement in company profits to compare to the previous model. 

In [11]:
items['rental_revenue'] = items.apply(lambda df: df['rental_count']*df['rental_price'],
                                      axis=1)

# Data Cleaning Pipeline

The data contains several samples that need to be removed either because there is not enough information for it to be useful or they need to be corrected in one way or another.

In [12]:
# Remove erroneous samples

items = items.query('brand!="LENDER SUBMISSION FILL IN"').copy()

items = items.query('rental_price != 999985.0').copy()

In [13]:
# Correct items with unrealistic rental prices

f = lambda df: 0.15*df['cost'] if df['rental_price']==1000000.0 else df['rental_price']

items['rental_price'] = items.apply(f, axis=1)

In order to determine the lifetime of items we need to know when they were delisted. But most items continue to be rented so in those cases we chose the current date as the date that they were delisted.

In [14]:
# items that have not been removed
# receive curent date as removal time

items['removed_at'] = items['removed_at'].fillna(value=pd.to_datetime('now'))

Brand names are curated to remove any variablity in spellings. This reduced the list of brand names by 30%.

In [15]:
# clean brand names

items['brand'] = items['brand'].apply(lambda s: s.lower()).apply(lambda s: s.strip(' '))\
                               .apply(lambda s: s.replace('  ', ' '))\
                               .apply(lambda s: s.replace(u'\xa0', u' '))

items['brand'] = items['brand'].apply(lambda s: brand_names[s] if s in brand_names else s)

# Store Final Data

The final step of the data cleaning pipeline is storing the results into cvs files for later use.

In [16]:
items.to_csv(path_or_buf='data/items.csv', sep=',', header=True,
             index=True, index_label=None, mode='w',
             line_terminator='\n')

In [17]:
orders.to_csv(path_or_buf='data/orders.csv', sep=',', header=True,
             index=True, index_label=None, mode='w',
             line_terminator='\n')